## Pogramación Lineal
### Arbitraje de Opciones

Este cuaderno muestra un ejemplo de como verificar si es posible realizar arbitraje en una cadena
de opciones call de un mismo vencimiento. Para ello se construye un modelo de programación lineal
que calcula el coste de construir una cartera que:
- de beneficio no negativo si el precio es 0
- de beneficio no negativo para cada uno de los strikes de la cadena
- de beneficio no negativo a partir del último strike (pendiente positiva)

In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp

In [3]:
def call_value(K, S):
    """
    calcula el beneficio de la call a vencimiento.
    Si el precio de la accion es mayor que el de ejercicio
    el beneficio es la diferencia.  Si el precio es menor
    vence sin valor
    """
    p = 0
    if S > K:
        p = S - K
    return p

In [4]:
print(call_value(100, 105))
print(call_value(100, 95))

5
0


### Datos
Presentamos como ejemplo un diccionario que representa la prima para cada precio de ejercicio
en una cadena de opciones 

In [5]:
strike_call_prices ={
    80: 20.14,
    85: 15.47,
    90: 11.23,
    95: 7.64,
    100: 4.85,
    105: 2.86,
    110: 1.58,
    115: 0.81,
    120: 0.39
}

In [6]:
# numero de ejercicios
n = len(strike_call_prices)
n

9

In [47]:
# array con los precios
call_prices = np.array(list(strike_call_prices.values()))
call_prices

array([20.14, 15.47,  9.23,  7.75,  5.03,  2.86,  1.05,  0.81,  0.39])

In [48]:
# array con los precios de ejercicio
call_strikes = list(strike_call_prices.keys())
call_strikes

[80, 85, 90, 95, 100, 105, 110, 115, 120]

### Modelo

Las variables de decisión representan el número de contratos que compraremos (positivo)
o venderemos (negativo) para nuestra cartera

In [49]:
# variables de decision
call_x = cp.Variable(n)

In [50]:
# precio de la cartera primas x contratos que queremos minimizar
objetivo = cp.sum(cp.multiply(call_x, call_prices))

In [51]:
#lista de restricciones
constraints = []

(1) añadir restricciones del valor a vencimiento si el precio es 0

In [52]:
call_strikes

[80, 85, 90, 95, 100, 105, 110, 115, 120]

In [53]:
value_at0 = np.zeros(n)
for i, i_strike in enumerate(call_strikes):
    value_at0[i] = call_value(i_strike, 0)
print(value_at0)
exp_at0 = cp.sum(cp.multiply(value_at0, call_x)) >= 0
constraints.append(exp_at0)

[0. 0. 0. 0. 0. 0. 0. 0. 0.]


(2) añadir restricciones del valor a vencimiento para cada strike

In [54]:
for j_price in call_strikes:
    value_ati = np.zeros(n)
    for i, i_strike in enumerate(call_strikes):
        value_ati[i] = call_value(i_strike, j_price)
    print(j_price, "->", value_ati)  
    ati_exp = cp.sum(cp.multiply(value_ati,call_x)) >= 0
    constraints.append(ati_exp)

80 -> [0. 0. 0. 0. 0. 0. 0. 0. 0.]
85 -> [5. 0. 0. 0. 0. 0. 0. 0. 0.]
90 -> [10.  5.  0.  0.  0.  0.  0.  0.  0.]
95 -> [15. 10.  5.  0.  0.  0.  0.  0.  0.]
100 -> [20. 15. 10.  5.  0.  0.  0.  0.  0.]
105 -> [25. 20. 15. 10.  5.  0.  0.  0.  0.]
110 -> [30. 25. 20. 15. 10.  5.  0.  0.  0.]
115 -> [35. 30. 25. 20. 15. 10.  5.  0.  0.]
120 -> [40. 35. 30. 25. 20. 15. 10.  5.  0.]


(3) añadir restricciones de la pendiente del ultimo strike

In [55]:
top_strike = call_strikes[-1]
value_attop = np.zeros(n)
for i, i_strike in enumerate(call_strikes):
    value_attop[i] = call_value(i_strike, top_strike + 1) - call_value(i_strike, top_strike)
print(value_attop)

exp_top = cp.sum(cp.multiply(value_attop, call_x)) >= 0
constraints.append(exp_top)

[1. 1. 1. 1. 1. 1. 1. 1. 1.]


Se restringe el numero de contratos vendidos. En la práctica estaríamos limitados
por el control de riesgo de nuestra cuenta.  En el caso hipotético de no restringirse
podríamos vender todo lo que quisieramos cuando haya arbitraje

In [56]:
constraints.append(call_x >= -80) 

Solucionamos el problema
- resultado 0 significa que no existe arbitraje. La cartera mas barata sería equivalente a no comprar nada
- resultado negativo significa que en lugar de pagar recibimos dinero por nuestra cartera

In [57]:
problem = cp.Problem(cp.Minimize(objetivo), constraints)
result = problem.solve()

In [58]:
np.round(result, 4)

-125.2

compra venta de contratos que tendríamos que realizar

In [59]:
res_serie = pd.Series(np.round(call_x.value), index=call_strikes)
res_serie

80      40.0
85     -80.0
90     120.0
95     -80.0
100    -80.0
105     80.0
110     40.0
115    -80.0
120     40.0
dtype: float64

___


### Ejercicio Propuesto

Ejecutar el modelo del ejemplo considerando ahora unos precios de prima alternativos. 
Verificar que en este caso sí existe arbitraje. ¿Qué cartera tendrías que construir?

In [20]:
strike_call_prices ={
    80: 20.14,
    85: 15.47,
    90: 11.23,
    95: 7.75,
    100: 5.03,
    105: 2.86,
    110: 1.05,
    115: 0.81,
    120: 0.39
}

In [46]:
strike_call_prices ={
    80: 20.14,
    85: 15.47,
    90: 9.23,
    95: 7.75,
    100: 5.03,
    105: 2.86,
    110: 1.05,
    115: 0.81,
    120: 0.39
}